In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import datetime
import os
import sys
import time

In [3]:
sys.path.append("/home/caleml/main-pe/")

In [4]:
from data.datasets.h36m import Human36M
from data.utils.data_utils import TEST_MODE, TRAIN_MODE, VALID_MODE
from data.loader import BatchLoader

from experiments.common import exp_init

In [5]:
from model import blocks
from model import layers
from model import losses
from model import config
from model import callbacks
from model.utils import pose_format, log

In [6]:
from model.networks.multi_branch_model import MultiBranchModel
from model.networks.mbm_vgg import MultiBranchVGGModel

In [7]:
from model.networks.mbm_reduced import MultiBranchReduced, MultiBranchStopped

In [8]:
from tensorflow.keras.callbacks import TensorBoard

# Dataset

In [9]:
# local loading
h36m_path = '/home/caleml/datasets/h36m'
h36m = Human36M(h36m_path, dataconf=config.human36m_dataconf, poselayout=pose_format.pa17j3d, topology='frames') 

In [ ]:
conf = {
    'exp_type': 'hybrid_vgg_TEST_NB',
    'dim': 3,
    'n_joints': 17,
    'pose_blocks': 2,
    'dataset_name': 'h36m',
    'batch_size': 8,
    'n_epochs': 60
}

In [10]:
conf = {
    'exp_type': 'hybrid_reduced128_stopped_TEST_NB',
    'dim': 3,
    'n_joints': 17,
    'pose_blocks': 2,
    'dataset_name': 'h36m',
    'batch_size': 8,
    'n_epochs': 60
}

In [15]:
model_folder = exp_init(conf)

Conducting experiment for 60 epochs and 2 blocks in folder /home/caleml/pe_experiments/exp_20190506_1835_hybrid_reduced128_stopped_TEST_NB__2b_bs8


In [ ]:
# validation dataset
h36m_val = BatchLoader(
    h36m, 
    ['frame'], 
    ['pose_w', 'pose_uvd', 'afmat', 'camera'], 
    VALID_MODE, 
    batch_size=h36m.get_length(VALID_MODE), 
    shuffle=True)

In [ ]:
log.printcn(log.OKBLUE, 'Preloading Human3.6M validation samples...')
[x_val], [pw_val, puvd_val, afmat_val, scam_val] = h36m_val[0]

In [ ]:
eval_callback = callbacks.H36MEvalCallback(conf['pose_blocks'], x_val, pw_val, afmat_val, puvd_val[:,0,2], scam_val, logdir=model_folder)

# Training

In [ ]:
# VGG with action for phony placeholders
data_tr_h36m = BatchLoader(
        h36m, 
        ['frame'], 
        ['action'] * 3 + ['pose'] * conf['pose_blocks'],
        TRAIN_MODE, 
        batch_size=conf['batch_size'],
        shuffle=True)

In [ ]:
# VGG loader with phony keyword
data_tr_h36m = BatchLoader(
        h36m, 
        ['frame'], 
        ['phony'] * 3 + ['pose'] * conf['pose_blocks'],
        TRAIN_MODE, 
        batch_size=conf['batch_size'],
        shuffle=True)

In [ ]:
# VGG loader with specific phony sizes
data_tr_h36m = BatchLoader(
        h36m, 
        ['frame'], 
        ['phony_2_b_256_256_64', 'phony_2_b_128_128_128', 'phony_2_b_64_64_256'] + ['pose'] * conf['pose_blocks'],
        TRAIN_MODE, 
        batch_size=conf['batch_size'],
        shuffle=True)

In [11]:
# classical i_hat + pose format
data_tr_h36m = BatchLoader(
        h36m, 
        ['frame'], 
        ['frame'] + ['pose'] * conf['pose_blocks'],
        TRAIN_MODE, 
        batch_size=conf['batch_size'],
        shuffle=True)

y_dict ['frame', 'pose', 'pose'], allkeys ['frame', 'frame', 'pose', 'pose']


In [ ]:
# classical + info for train eval
data_tr_h36m = BatchLoader(
        h36m, 
        ['frame'], 
        ['frame'] + ['pose'] * conf['pose_blocks'] + ['pose_w', 'afmat', 'pose_uvd', 'camera'],
        TRAIN_MODE, 
        batch_size=conf['batch_size'],
        shuffle=True)

### Classical multi branch

In [ ]:
model = MultiBranchModel(dim=conf['dim'], n_joints=conf['n_joints'], nb_pose_blocks=conf['pose_blocks'])
model.build()

In [ ]:
# model.add_callback(eval_callback)

In [ ]:
model.train(data_tr_h36m, steps_per_epoch=len(data_tr_h36m), model_folder=model_folder, n_epochs=conf['n_epochs'], cb_list=[])

In [ ]:
# short test for cb
model.train(data_tr_h36m, steps_per_epoch=10, model_folder=model_folder, n_epochs=conf['n_epochs'])

### VGG multi branch

In [ ]:
model = MultiBranchVGGModel(dim=conf['dim'], n_joints=conf['n_joints'], nb_pose_blocks=conf['pose_blocks'])
model.build()

In [ ]:
model.train(data_tr_h36m, steps_per_epoch=len(data_tr_h36m), model_folder=model_folder, n_epochs=conf['n_epochs'], cb_list=[])

### Reduced multi branch

In [ ]:
from model.networks.mbm_reduced import MultiBranchReduced
model = MultiBranchReduced(dim=conf['dim'], n_joints=conf['n_joints'], nb_pose_blocks=conf['pose_blocks'])
model.build()

In [ ]:
cb_list = []

In [ ]:
logs_folder = os.environ['HOME'] + '/pe_experiments/tensorboard/' + model_folder.split('/')[-1]
print('Tensorboard log folder %s' % logs_folder)
tensorboard = TensorBoard(log_dir=os.path.join(logs_folder, 'tensorboard'))

In [ ]:
eval_callback = callbacks.H36MEvalCallback(conf['pose_blocks'], x_val, pw_val, afmat_val, puvd_val[:,0,2], scam_val, pose_only=False, logdir=model_folder)

In [ ]:
cb_list.append(tensorboard)
cb_list.append(eval_callback)
# cb_list.append(LearningRateScheduler(lr_scheduler))
cb_list.append(callbacks.SaveModel(model_folder))

In [ ]:
# train with CB
model.train(data_tr_h36m, steps_per_epoch=10, model_folder=model_folder, n_epochs=conf['n_epochs'], cb_list=cb_list)

In [ ]:
# train without CB
model.train(data_tr_h36m, steps_per_epoch=100, model_folder=model_folder, n_epochs=conf['n_epochs'], cb_list=[])

### Reduced stopped

In [13]:
model = MultiBranchStopped(dim=conf['dim'], n_joints=conf['n_joints'], nb_pose_blocks=conf['pose_blocks'])
model.build()

reshaping via a convolution...
reshaping via a convolution...
reshaping via a convolution...
reshaping via a convolution...
zp_depth 128
pose shape (?, 17, 3), vis shape (?, 17, 1), concat shape (?, 17, 4)
pose shape (?, 17, 3), vis shape (?, 17, 1), concat shape (?, 17, 4)
Last H shape Tensor("fReMap2_1/batch_normalization_83/cond/Merge:0", shape=(?, 32, 32, 576), dtype=float32)
Build E_a 2.208324909210205, build E_p 13.295784711837769, decoder D 1.1317250728607178
Input shape (?, 256, 256, 3)
Shape z_a (?, 16, 16, 128), shape z_p (?, 16, 16, 128)
Outputs shape [(None, 256, 256, 3), (None, 17, 4), (None, 17, 4)]
rec y_pred shape (?, 256, 256, 3)
Final model summary
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
image_input (InputLayer)        (None, 256, 256, 3)  0                                            
____________________________

In [16]:
model.train(data_tr_h36m, steps_per_epoch=100, model_folder=model_folder, n_epochs=conf['n_epochs'], cb_list=[])

Training with 0 callbacks
Epoch 1/60
100/100 [==============================] - 34s 336ms/step - loss: 12.3587 - decoder_loss: 7.5601 - pose_model_loss: 2.3361
Epoch 2/60
100/100 [==============================] - 18s 180ms/step - loss: 6.7418 - decoder_loss: 3.0699 - pose_model_loss: 1.7440
Epoch 3/60
100/100 [==============================] - 18s 177ms/step - loss: 4.8352 - decoder_loss: 1.4812 - pose_model_loss: 1.5811
Epoch 4/60
100/100 [==============================] - 17s 171ms/step - loss: 4.0238 - decoder_loss: 0.9269 - pose_model_loss: 1.4608
Epoch 5/60
100/100 [==============================] - 17s 174ms/step - loss: 3.8125 - decoder_loss: 0.8117 - pose_model_loss: 1.4131
Epoch 6/60
  6/100 [>.............................] - ETA: 15s - loss: 3.8454 - decoder_loss: 1.0853 - pose_model_loss: 1.2971

KeyboardInterrupt: 

# VGG debug stuff

In [ ]:
from tensorflow.keras.applications import VGG16

In [ ]:
vgg_model = VGG16(include_top=False, weights='imagenet', input_shape=(256, 256, 3))

In [ ]:
vgg_model.summary()

In [ ]:
output_layers = [1,3,4,6,7]
for i in output_layers:
    print(vgg_model.layers[i].name)

In [ ]:
# conv1_1,conv2_1,conv3_1,pool1,pool2
# from https://discuss.pytorch.org/t/how-to-use-vgg-19-network-to-estimate-perceptual-loss/9981

In [ ]:
a = 'phony_2_b_256_256_64'
batch_size = 24
b = [int(elt) if elt.lower() != 'b' else batch_size for elt in a.split('_')[1:]]

In [ ]:
b

In [ ]:
def foo(a, b, c):
    
    def loss(y_true, y_pred):
        return y_true + y_pred + a + b + c
    
    return loss
    

In [ ]:
loss = foo(1, 2, 3)

In [ ]:
print(type(loss))
loss(10, 10)

## Dataset debug

In [ ]:
import scipy.io as sio
annot_file = os.path.join(h36m_path, 'annotations.mat')
mat = sio.loadmat(annot_file, struct_as_record=False, squeeze_me=True)

In [ ]:
sequences = [mat['sequences_te'], mat['sequences_tr'], mat['sequences_val']]
action_labels = mat['action_labels']
joint_labels = mat['joint_labels']

In [ ]:
type(joint_labels)
joint_labels.shape

In [ ]:
# Human 3.6 -> PA17J
joint_labels[[0, 12, 13, 15, 25, 17, 26, 18, 27, 19, 1, 6, 2, 7, 3, 8, 11]]

In [ ]:
print(type(sequences[1]))
sequences[1][0].shape

In [ ]:
import numpy as np
a = np.array([[1,2,3], [4,5,6]])
b = np.reshape(a, 6)

print(a)
print(b)